# Imports & Settings


In [41]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [115]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})

# Splitting

In [116]:
data = combined_data_fully_translated

In [83]:
data.country_and_unit.value_counts()

                             8212
Eesti                        2232
Eesti  Tallinn               1320
Eesti  Tartu                  185
Saksamaa                      172
Vene                          170
NSVL                          143
Eesti  Pärnu                   83
Rootsi                         68
Soome                          45
Eesti  Viljandi                44
Inglismaa                      42
Läti                           41
Ameerika Ühendriigid           38
Eesti  Haapsalu                33
Eesti  Narva                   29
Venemaa                        26
Eesti  Kõmsi                   22
Eesti  Kohtla-Järve            18
Eesti  Kesklinna               17
Eesti  Rakvere                 17
Prantsusmaa                    16
Eesti  Harju                   15
Austria                        14
Kasahstan                      12
Kanada                         11
Eesti  Narva-Jõesuu            11
Eesti  Kuressaare              10
Leedu                           9
Itaalia       

In [109]:
def extract_city_country(item):
    if item == None:
        return [float('nan'), float('nan')]
        
    if (type(item) == str):
        item = item.strip()
        item = re.sub(' +', ' ', item) # remove multiple spaces

        if (' ' in item) and ('Eesti' in item):        
            split = item.split(' ')
            return [' '.join(split[1:]), split[0]]
        else: 
            return [float('nan'), item]
    else:
        return [float('nan'), float('nan')]

In [110]:
data['city'] = data.apply(lambda item: extract_city_country(item['country_and_unit'])[0], axis=1)
data['country'] = data.apply(lambda item: extract_city_country(item['country_and_unit'])[1], axis=1)

In [120]:
train.head()

,full_nr,name,ks,material,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,technique,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
232170,ETMM _ 12150:115 Aj 118:44/M20,"Kuno Areng, Bremerhaveni Festwoche medal",118.0,metall,NaN,festivalid,linn Bremerhaven,1979,NaN,ei,Saksamaa,osaleja,"Areng, Kuno",NaN,NaN,NaN,NaN,läbimõõt,cm,4,ETMM,_,12150.0,115.0,NaN,Aj,44.0,M20,1.0,NaN,1.0,Festwoche- Breemenhaven,NaN,hea,hall,KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E. V.,medal
2251378,ETMM _ 12584:19 M 102:1/13:13,"Foto-Villem Kapp,foto pühendusega Armilde M,1937",102.0,fotomaterjal,NaN,filmindus ja fotograafia,NaN,1938,NaN,ei,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12584.0,19.0,NaN,M,1.0,13:13,1.0,"Fotod Villem Kapi ja Juhan Aaviku kogu-\ndesse, ostetud 2013. aastal",NaN,NaN,NaN,hea,NaN,NaN,foto
2070466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,F,52.0,NaN,1.0,NaN,NaN,NaN,NaN,hea,NaN,NaN,foto
4085096,ETMM _ Fk 41691/k,"Metspart, Noorsooteater, 1969, osades: Hedvig - Mari Lill, Gina - Silvia Laidla",41691.0,fotomaterjal,NaN,teater,NaN,18.09.1969,NaN,ei,,seosorganisatsioon,Noorsooteater,NaN,NaN,NaN,NaN,filmikaader,60 x 60 mm,1,ETMM,_,NaN,NaN,NaN,Fk,NaN,k,1.0,NaN,1.0,TB080321,NaN,rahuldav,NaN,NaN,fotonegatiiv
2697904,ETMM _ 9424 Mo 238:1/62:05,Kiri: Rahvapillimehed: Viiul: Jüri Saal: Kiri A. Pulstile: 16.02.1936,238.0,paber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,käsikiri,NaN,NaN,NaN,ETMM,_,9424.0,NaN,NaN,Mo,1.0,62:05:00,1.0,Rahvapillimehed Mo238,1.0,NaN,NaN,rahuldav,NaN,NaN,kiri


In [ ]:
data.